In [1]:
from loader.daily_loader import load_daily_paper
from chater.chat import mapreduce
import os
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import faiss
import json

from vec_generate.arxiv_generate import arxiv_generate

In [2]:
#初始化部分
os.environ["NVIDIA_API_KEY"] = "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"
model = ChatNVIDIA(model="ai-llama3-8b")#.bind(max_tokens=4096)
embedder = NVIDIAEmbeddings(model="ai-embed-qa-4")
cache = {}
try:
    cache = json.load(open("cache.json"))  # 读取用户有过哪些向量库   里面存的应该是tuple (vectorstore_path, initial_msg)
except:
    cache = {}
    json.dump(cache, open("cache.json", "w"))

In [3]:
embedder.embed_documents(["hello world"])  # 预热

[[0.01036834716796875,
  -0.05035400390625,
  0.0197906494140625,
  0.02587890625,
  -0.0020599365234375,
  -0.00244903564453125,
  -0.07769775390625,
  -0.0034027099609375,
  -0.013153076171875,
  -0.0299530029296875,
  0.059112548828125,
  0.03472900390625,
  0.06781005859375,
  0.063720703125,
  -0.01519012451171875,
  0.02301025390625,
  0.034576416015625,
  -0.0166015625,
  0.01313018798828125,
  -0.0311126708984375,
  0.044769287109375,
  -0.031280517578125,
  -0.02642822265625,
  0.0024738311767578125,
  0.00304412841796875,
  -0.0260467529296875,
  -0.0208587646484375,
  -0.003238677978515625,
  -0.038604736328125,
  0.03668212890625,
  0.0009522438049316406,
  -0.040802001953125,
  -0.0223846435546875,
  -0.00519561767578125,
  0.041717529296875,
  -0.0017633438110351562,
  0.018768310546875,
  -0.02362060546875,
  0.004482269287109375,
  0.030517578125,
  0.006565093994140625,
  0.0123748779296875,
  0.0298614501953125,
  0.0618896484375,
  -0.0172119140625,
  0.0191650390625

这里先不给用户造自己的库的功能 只有daily chat和 awesome chat 两个功能  本地有个csv表格记录已经做了哪些库

In [4]:


def daily_chat(date, cat, model, embedder, cache={}, verbose=False):

    '''
    接受日期 类别 模型和cache, 返回一个向量库路径和一个开场词
    如果用户指定verbose为True, 则当天的arxiv表格会保存在该库本地
    '''

    if date+cat in cache:
        return cache[date+cat]
    
    else:
        daily = load_daily_paper(date)

        if(len(daily) == 0):
            print("No papers found for this date and category, please retry")
            return None, None

        summary, detail = mapreduce(model, daily)
        if verbose:
            daily.to_csv(f"{date}_{cat}.csv")  # 保存当天的arxiv表格
        
        path_name = 'local_embed/'+date+cat
        arxiv_generate(detail['id'], embedder, path_name)
        
        cache[date+cat] = (path_name, summary)
        json.dump(cache, open("cache.json", "w"))

        return path_name, summary
        
        

In [5]:
from loader.awesome_loader import load_from_awesome
def awesome_chat(md_path, embedder, cache={}):
    '''
    接受md路径和cache, 返回一个向量库路径和一个开场词
    '''
    if md_path in cache:
        return cache[md_path]
    else:
        with open(md_path, 'r') as f:
            text = f.read()
            
        path_name = 'local_embed/'+md_path.split('.')[0]
        
        paper_list = load_from_awesome(text)

        msg = (
    "你好，我是awesome-parser， 一个帮助用户解析awesome-list的工具。"
    f"我从您提供的文档{md_path}中解析到了{len(paper_list)}篇paper，我可以帮助您吗")

        arxiv_generate(paper_list[:2], embedder, path_name)
        cache[md_path] = (path_name, msg)
        json.dump(cache, open("cache.json", "w"))

        return path_name, msg
    

In [6]:
path,init_msg1 = daily_chat("20240707", "cs.CR", model, embedder, cache, verbose=True)

In [7]:
path, init_msg2 = awesome_chat("prompt_injection.md", embedder, cache)

In [8]:
print(path,init_msg1)

local_embed/prompt_injection Based on the summaries of the articles, I can identify some trends and potential research questions in the following areas:

1. **Privacy and Gradient Protection in Federated Learning**:
	* Trend: Researchers are focusing on protecting the privacy of gradients in federated learning, as existing methods may not provide sufficient protection.
	* Potential research questions:
		+ How can we improve the robustness of gradient protection methods against perturbations and attacks?
		+ Can we develop new methods to recover perturbed gradients without accessing the original model structure or third-party data?
2. **IoT Security and Context-Aware Sharing Platforms**:
	* Trend: The importance of securing IoT deployments and critical infrastructure is increasingly recognized, and researchers are exploring novel solutions for real-time threat modeling and mitigation.
	* Potential research questions:
		+ How can we develop more effective and efficient methods for threat

In [9]:
vecstores = [FAISS.load_local(folder_path=path, embeddings=embedder,allow_dangerous_deserialization=True)]

In [10]:
from vec_generate.arxiv_generate import aggregate_vstores

docstore = aggregate_vstores(vecstores, embedder)
print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 288 chunks


In [11]:
from vec_generate.arxiv_generate import default_FAISS
convstore = default_FAISS(embedder)

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from functools import partial
from langchain.document_transformers import LongContextReorder
from operator import itemgetter
from langchain_core.runnables.passthrough import RunnableAssign

    
def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: \n\nHow can I help you?"
)


chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.Reply must more than 100 words)"
), ('user', '{input}')])

## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## 将较长的文档重新排序到输出文本的中心， RunnableLambda在链中运行无参自定义函数 ，长上下文重排序（LongContextReorder）
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

retrieval_chain = (
    {'input' : (lambda x: x)}
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever()  | long_reorder | docs2str})
    | RPrint()
)
stream_chain = chat_prompt | model | StrOutputParser()

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ##首先根据输入的消息进行检索
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## 然后流式传输stream_chain的结果
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## 优化信息打印的格式
        if not return_buffer:
            line_buffer += token
            if "\n" in line_buffer:
                line_buffer = ""
            if ((len(line_buffer)>84 and token and token[0] == " ") or len(line_buffer)>100):
                line_buffer = ""
                yield "\n"
                token = "  " + token.lstrip()
        yield buffer if return_buffer else token

    ##最后将聊天内容保存到对话内存缓冲区中
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)

In [43]:
import gradio as gr
import re

def download_file(textbox1,textbox2):  
       #下载函数，用于下载csv文件
       #_,_ ,cvspath= daily_chat(textbox1, textbox2, model, embedder, cache, verbose=True)
       return  '20240707_cs.CR.csv'
def separate_numbers_and_letters(text):
    # 使用正则表达式匹配数字和字母，将需要的文本内容进行拆分
    numbers = re.findall(r'\d+', text)
    letters = re.findall(r'[a-zA-Z]+', text)
    return numbers, letters
#三个预设的左侧chatbot，只用来输出默认的预设对话和提供组件框
chatbot1_init = gr.Chatbot(height= 500,value=[[None,initial_msg]]) 
chatbot2_init = gr.Chatbot(height= 500,value=[[None,initial_msg]]) 
chatbot3_init = gr.Chatbot(height= 500,value=[[None,initial_msg]]) 
def show_popup():
    return gr.update(visible=True)
with gr.Blocks(theme=gr.themes.Soft(),fill_height = True ) as demo:
    with gr.Tab(label="arxiv"):
        
        with gr.Row():#左侧的功能区
            with gr.Column(scale=4,min_width= 150):
                textbox1= gr.Textbox(placeholder="查看日期", container= False, scale= 3)
                textbox2= gr.Textbox(placeholder="查看类型", container= False, scale= 3)
                download = gr.DownloadButton(label="Download CSV")
                download.click(download_file, [textbox1,textbox2], [download])
            with gr.Column(scale=10,min_width= 300):#左侧的预设区，输入对应日期+种类即可查询，不过对于文本的格式的读取有问题，
                      intput_text1 = gr.Textbox(placeholder="查看当天的arxiv", container= False, scale= 10)
                      @gr.render(inputs= intput_text1,triggers=[intput_text1.submit])
                      def show_rebot(text):
                            if len(text) == 0:
                                gr.Markdown(f"<h3 style='text-align: center; margin-bottom: 1rem'>请输入想要查询的arxiv的日期</h3>")
                            else:
                                chatbot_fr1 = gr.Chatbot(height= 500,value=[[None,daily_chat("20240707", "cs.CR", model, embedder, cache, verbose=False)[1]]]) 
            with gr.Column(scale=20,min_width=600):#右侧的对话区
                        gr.ChatInterface(
                                fn=chat_gen,
                                chatbot=chatbot1_init, 
                                additional_inputs=None,
                                title="每日arxiv",
                                submit_btn="发送",
                                autofocus= True,
                                fill_height = True,
                                undo_btn= "↩️ 清空前言",
                                clear_btn= "🗑️ 清空对话",
                                stop_btn= "⏸ 停止生成",
                                retry_btn= "🔄  重新生成",
                        
                            ).queue()
    with gr.Tab(label="awsome"):
        with gr.Row(): 
            with gr.Column(scale=2,min_width= 100):#左侧的预设区，上传文件后即可对上传的文件利用awesome_chat进行赏析，不过filemd输出的文件路径不清楚
                gr.Markdown(f"<h3 style='text-align: center; margin-bottom: 1rem'>请上传查询的md</h3>")
                file_md = gr.File(file_count='single',label="put markdown file here", file_types=["md"])  
                @gr.render(inputs= [file_md])
                def awsomebot(file):
                    if file:
                        chatbot_fr3 = gr.Chatbot(height= 500,value=[[None,awesome_chat("prompt_injection.md", embedder, cache)[1]]])            
                    else:
                        pass
            with gr.Column(scale=6):#右侧的对话区
                gr.ChatInterface(
                                fn=chat_gen,
                                chatbot=chatbot3_init,
                                additional_inputs=None,
                                multimodal= True,
                                title="awsome",
                                submit_btn="Submit",
                                fill_height = True,
                                undo_btn= "↩️ 清空前言",
                                clear_btn= "🗑️ 清空对话",
                                stop_btn= "⏸ 停止生成",
                                retry_btn= "🔄  重新生成",                        
                    ).queue()
                        

try:
    demo.launch(debug=True, share=True, show_api=True, server_port=5000, server_name="0.0.0.0")
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e


Running on local URL:  http://0.0.0.0:5000
Running on public URL: https://066f299f4b67a0ad51.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:5000 <> https://066f299f4b67a0ad51.gradio.live
Closing server running on port: 5000
